In [ ]:
!pip install flask
!pip install pyngrok
!pip install flask_cors

In [ ]:
#mount drive (do not have to do)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import GRU, Dropout, LSTM
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Embedding, Reshape, Activation, Input, Dense, Masking, Conv1D, Bidirectional,Flatten
from tensorflow.python.keras.layers.merge import Dot
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras.utils.data_utils import get_file
from tensorflow.python.keras.utils.np_utils import to_categorical
from tensorflow.keras.preprocessing.sequence import skipgrams
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import json

In [ ]:
max_word = 3458
num_word = 30
max_len = 45

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/idx_to_word.json') as json_file:
    idx = json.load(json_file)
with open('/content/drive/MyDrive/Colab Notebooks/word_to_idx.json') as json_file:
    word = json.load(json_file)

In [ ]:
idx_to_word = dict()

for i in idx.keys():
  idx_to_word[int(i)] = idx[i]


In [ ]:
word_to_idx= dict()

for i in word.keys():
  word_to_idx[i] = int(word[i])

In [ ]:
word_to_idx['<MASK>'] = 0
idx_to_word[40] = 'แต่'
word_to_idx['แต่'] = 40

In [ ]:
idx_to_word[0] = '<MASK>'

In [ ]:
input = Input(shape = (max_len,))
embedding = Embedding(max_word+1, 256,mask_zero=True,trainable = True)(input)
gru = GRU(32)(embedding)
dropout = Dropout(0.5)(gru)
output = Dense(2, activation='softmax')(dropout)
cls_model = Model(input,output)
opt=Adam(lr=0.01)
cls_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
cls_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 45)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 45, 256)           885504    
_________________________________________________________________
gru (GRU)                    (None, 32)                27840     
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 2)                 66        
Total params: 913,410
Trainable params: 913,410
Non-trainable params: 0
_________________________________________________________________


In [ ]:
cls_model.load_weights("/content/drive/MyDrive/Colab Notebooks/model_weight_lda.h5")

In [ ]:
def get_model():

  input1 = Input(shape=(num_word,))
  emb_out = (Embedding(len(word_to_idx)+1,128,input_length=num_word,trainable=True,mask_zero=True))(input1)

  token_emb_out = Embedding(len(word_to_idx)+1,128,input_length=num_word,mask_zero=True)(input1)
  x = Bidirectional(GRU(64, return_sequences=True))(token_emb_out)
  x = Flatten()(x)
  x = Dropout(0.2)(x)
  out = Dense(len(word_to_idx), activation='sigmoid')(x)

  model = Model(inputs=input1, outputs=out)
  model.compile(optimizer=Adam(),
                  loss='categorical_crossentropy',
                  metrics=['acc'])
  return model

model = get_model()
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 30)]              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 30, 128)           358528    
_________________________________________________________________
bidirectional (Bidirectional (None, 30, 128)           74496     
_________________________________________________________________
flatten (Flatten)            (None, 3840)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 3840)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2800)              10754800  
Total params: 11,187,824
Trainable params: 11,187,824
Non-trainable params: 0
_______________________________________________

In [ ]:
model.load_weights("/content/drive/MyDrive/Colab Notebooks/text_gen_7.h5")

In [ ]:
def convert_to_input(seq,lenght):
  x_input = []

  if (len(seq)<lenght):
    x_input.extend(seq + ["<MASK>"]*(lenght-len(seq)))
  else:
    for i in range(len(seq)-lenght, len(seq)):
      x_input.append(seq[i])
  return x_input

In [ ]:
import random
import time

In [ ]:
def current_milli_time():
    return round(time.time() * 1000)

In [ ]:
def word2features(word, word_dict_to_idx):
    if word in word_dict_to_idx :
        return word_dict_to_idx[word]
    else :
        return word_dict_to_idx['UNK']

def sent2features(sentence,word_dict_to_idx):
    return np.asarray([word2features(word,word_dict_to_idx) for word in sentence])

def sent2tokens(sent):
    return [word for (word, label) in sent]

In [ ]:
def gen_text(model, seed_word, lenght, temp):
  seed_text = seed_word
  for i in range(lenght):
    x_input = convert_to_input(seed_text, num_word)
    x_i = np.array([sent2features(x_input,word_to_idx)])
    y_pred=model.predict(x_i)
    weight = y_pred[0]
    tmp_probs = []
    sum_tmp_probs = 0
    for i in range(len(weight)):
      val = weight[i] ** (1/temp)
      tmp_probs.append(val)
      sum_tmp_probs += val
    for i in range(len(tmp_probs)):
      tmp_probs[i] /= sum_tmp_probs

    random.seed(current_milli_time())
    sampled_number = random.random()

    current = 0
    max_idx = 0
    for idx, val in enumerate(tmp_probs):
      if current + val > sampled_number:
        max_idx = idx
        break
      current += val
    
    if (idx_to_word[max_idx] == "<E>"):
      seed_text.append(idx_to_word[max_idx])
      break
    seed_text.append(idx_to_word[max_idx])

  return seed_text

In [ ]:
def prediction(topic,seedtext):
  text_array = []
  index_text = []
  for i in range(5):
    text_array.append(gen_text(model, ["<S>",seedtext], 25, 0.0001))
  for i in range(len(text_array)):
    sen = []
    for word in text_array[i]:
      sen.append(word_to_idx[word])
    index_text.append(sen)
  index_text = sequence.pad_sequences(index_text)
  max_pred = 0
  max_position = 0
  for i in range(len(index_text)):
    y_pred = cls_model.predict(index_text[i])
    if np.argmax(y_pred, axis=0)[topic] > max_pred:
      max_position = i
  return "".join(text_array[max_position])    

In [ ]:
import os
import threading

from flask import Flask,request,jsonify
from pyngrok import ngrok
from flask_cors import CORS
import pickle

os.environ["FLASK_ENV"] = "development"

app = Flask(__name__)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'
app.config['Content-type'] = 'application/json'
app.config['Accept'] = 'application/json'

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(5000).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}/\"".format(public_url, 5000))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# Update inbound traffic via APIs to use the public-facing ngrok URL
# Define Flask routes
@app.route("/api")
def index():
    
    sentence = request.args.get('sentence')
    topic = request.args.get('topic')

    a = prediction(int(topic),sentence)
    
    return jsonify(output = a)

# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()

 * ngrok tunnel "http://080d037fa3df.ngrok.io" -> "http://127.0.0.1:5000/"
 * Serving Flask app "__main__" (lazy loading)
 * Environment: development


 * Debug mode: on
